# Zagon robota

In [ ]:
import rospy

In [ ]:
ns = "pingvin_2"
tool_name = "gripper"

In [ ]:
rospy.init_node(ns)

In [ ]:
from robotblockset.ros.franka import panda_ros

In [ ]:
r = panda_ros(ns=ns, control_strategy="JointImpedance", init_node=False)

# Zajem trajektorije

In [ ]:
import numpy as np
import time

def record_robot_state(robot, frequency, duration):
    interval = 1.0 / frequency
    
    N=int(frequency*duration)
    tt = np.zeros((N,1))
    qt = np.zeros((N,r.nj))
    dqt = np.zeros((N,r.nj))

    i = 0
    start_time = time.monotonic()
    end_time = start_time + duration
    
    last_update = time.monotonic()

    while time.monotonic() <= end_time+interval and i < N:
        t = time.monotonic()
        if t - last_update >= interval:
            
            # Get robot state
            state = robot.GetState()
            
            tt[i] = t-start_time
            qt[i] = r.q
            dqt[i] = r.qdot

            # Update the last recorded time
            last_update = t
            i += 1
        
    return tt, qt, dqt, i, N    

In [ ]:
#Robot naj bo podajen
r.SetJointCompliant()

In [ ]:
## ZAJEM

In [ ]:
import time
from utils import pip

time.sleep(2)
pip();
tt, qt, dqt, i, N = record_robot_state(r,100,3)
pip();

In [ ]:
from robotblockset.graphics import plotjtraj
import matplotlib.pyplot as plt

fig, ax = plt.subplots(3, 1, figsize=(9, 6))
fig.suptitle('Joint trajectories')
plotjtraj(tt, qt, dqt, ax=ax);
plotjtraj(tt, qt+0.01, dqt, ax=ax, linestyle="--");

Robota nredimo spet togega

In [ ]:
import time
def SoftSetJointCompliance(robot, target_k, tm):
    # Get the current joint stiffness values
    k0 = robot.joint_compliance.K
    
    # Calculate the number of steps
    N = max(1, round(tm / robot.tsamp))
    
    # Calculate the incremental changes 
    dK = (target_k - k0) / N

    #initialize
    Ki = k0
    target_d = robot._franka_default.JointCompliance.D

    robot.controller._verbose = 0
    # Gradually update compliance
    for i in range(1, N):
        Ki += dK
        robot.SetJointCompliance(Ki,target_d)
        time.sleep(robot.tsamp)
        
    robot.controller._verbose = 1
    robot.SetJointCompliance(Ki+dK,target_d)

In [ ]:
r.ErrorRecovery()

In [ ]:
r.SetJointCompliant()

In [ ]:
r.ResetCurrentTarget()
SoftSetJointCompliance(r,r._franka_default.JointCompliance.K,3);

# DMP

In [1]:
from dmp import DMP

**POZOR! Pri klicu objekta DMP kot argument podajte KOPIJE vhodnih spremenljivk!**

In [2]:
d = DMP(qt.copy(),np.squeeze(tt.copy()),vel_data=dqt.copy(),num_weights=25)

NameError: name 'qt' is not defined

In [ ]:
trj_dec,t_dec=d.decode()

In [ ]:
plotjtraj(qt=trj_dec, t=t_dec);

Funkcija `robot.JPath(path,t)` najprej premakne robota za prvo točko poti z linearnim gibom v kartezičnem prostoru.

Potem nadaljuje z izvajanjem dane poti v sklepnih koordinatah (`path`), pri čemer se robot v posamezne konfiguracije premakne v časih podanih v ločenem vektorju (`t`).

Alternativno lahko podamo tudi samo čas v katerem naj robot izvede celoten gib. 


In [ ]:
pip();
r.JPath(trj_dec,t_dec*3)

In [ ]:
trj_dec[1]

In [2]:
# Move slowly to the first configuration
r.JMove(trj_dec[0],5)
pip();
for q in trj_dec: 
    # Call the GoTo_q function
    r.GoTo_q(q,np.zeros(7),np.zeros(7),0)
    time.sleep(1/100.)

NameError: name 'r' is not defined

In [ ]:
# Jedrne funkcije
import numpy as np
import matplotlib.pyplot as plt

x = np.exp(-d.a_x * d._time_vec / d.tau)
psi = np.zeros((x.size, d.num_weights))
i = 0
for val in x:
    psi[i] = np.exp(- np.square(val-d.c) /
                 (np.multiply(d.sigma, 2)))
    i+=1

plt.plot(psi)
plt.show()

In [ ]:
fx = np.zeros(''' število samplov ''')
i=0
for x_, psi_ in zip(x, psi):
    fx[i] = sum(np.multiply(
        (np.multiply(d.weights_pos[], x_)),
        (np.divide(psi_, sum(psi_)))
    ))
    i+=1